In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
final=pd.read_csv('/content/drive/MyDrive/airline_sentiment_analysis.csv')

In [ ]:
final.head()

,Unnamed: 0,airline_sentiment,text
0,1,positive,@VirginAmerica plus you've added commercials t...
1,3,negative,@VirginAmerica it's really aggressive to blast...
2,4,negative,@VirginAmerica and it's a really big bad thing...
3,5,negative,@VirginAmerica seriously would pay $30 a fligh...
4,6,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [ ]:
final.describe()

,Unnamed: 0
count,11541.000000
mean,7456.888311
std,4298.601764
min,1.000000
25%,3614.000000
50%,7601.000000
75%,11221.000000
max,14638.000000


In [ ]:
final.isna().sum()

Unnamed: 0           0
airline_sentiment    0
text                 0
dtype: int64

In [ ]:
final = final.drop('Unnamed: 0', axis = 1)

In [ ]:
final.head()

,airline_sentiment,text
0,positive,@VirginAmerica plus you've added commercials t...
1,negative,@VirginAmerica it's really aggressive to blast...
2,negative,@VirginAmerica and it's a really big bad thing...
3,negative,@VirginAmerica seriously would pay $30 a fligh...
4,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [ ]:
final.shape

(11541, 2)

In [ ]:
X = final['text']
y = final['airline_sentiment']

In [ ]:
import re

In [ ]:
text = np.array(final['text'])
text = text.flatten()

In [ ]:
for i in range(len(text)):
  text[i] = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text[i])
  text[i] = re.sub("(@[A-Za-z0-9_]+)", "", text[i])
  text[i] = re.sub("(#[A-Za-z0-9_]+)", "", text[i])


In [ ]:
text = np.array(text)
X = pd.DataFrame(text, columns=['text'])

In [ ]:
X.head()

,text
0,plus you've added commercials to the experien...
1,"it's really aggressive to blast obnoxious ""en..."
2,and it's a really big bad thing about it
3,seriously would pay $30 a flight for seats th...
4,"yes, nearly every time I fly VX this “ear wor..."


In [ ]:
X.head()

,text
0,plus you've added commercials to the experien...
1,"it's really aggressive to blast obnoxious ""en..."
2,and it's a really big bad thing about it
3,seriously would pay $30 a flight for seats th...
4,"yes, nearly every time I fly VX this “ear wor..."


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 21)

In [ ]:
pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

lr = Pipeline([('vectorize', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])

lr.fit(X_train.flatten(),y_train.flatten())
y_pred1 = lr.predict(X_test.flatten())

print(accuracy_score(y_pred1,y_test.flatten()))

0.9094846253789519


In [ ]:
print(classification_report(y_pred1,y_test.flatten()))
print(confusion_matrix(y_pred1,y_test.flatten()))

              precision    recall  f1-score   support

    negative       0.99      0.91      0.95      2008
    positive       0.60      0.91      0.72       301

    accuracy                           0.91      2309
   macro avg       0.79      0.91      0.84      2309
weighted avg       0.94      0.91      0.92      2309

[[1825  183]
 [  26  275]]


In [ ]:
from xgboost import XGBClassifier

xgboost = Pipeline([('vectorizer', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])
xgboost.fit(X_train.flatten(), y_train.flatten())

y_pred2 = xgboost.predict(X_test.flatten())

print(accuracy_score(y_pred2,y_test.flatten()))

0.8895625812039845


In [ ]:
tfidf_vectorizer = TfidfVectorizer() 

tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train.flatten())
joblib.dump(tfidf_vectorizer.vocabulary_,open("feature.pkl","wb"))
tfidf_test_vectors = tfidf_vectorizer.transform(X_test.flatten())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
classifier = RandomForestClassifier()

classifier.fit(tfidf_train_vectors,y_train)
y_pred3 = classifier.predict(tfidf_test_vectors)

In [ ]:
print(classification_report(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))

              precision    recall  f1-score   support

    negative       0.89      0.99      0.93      1851
    positive       0.90      0.50      0.64       458

    accuracy                           0.89      2309
   macro avg       0.89      0.74      0.79      2309
weighted avg       0.89      0.89      0.88      2309

[[1825   26]
 [ 228  230]]


In [ ]:
pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.svm import SVC
import pickle
clf = SVC()
clf.fit(tfidf_train_vectors,y_train)
import joblib
filename = 'classifier.joblib.pkl'
_ = joblib.dump(clf, filename, compress=9)

y_pred = clf.predict(tfidf_test_vectors)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

    negative       0.92      0.98      0.95      1851
    positive       0.89      0.64      0.75       458

    accuracy                           0.91      2309
   macro avg       0.91      0.81      0.85      2309
weighted avg       0.91      0.91      0.91      2309

[[1816   35]
 [ 164  294]]


In [ ]:
clf2 = joblib.load(filename)
